In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Kodiak,US,57.79,-152.41,62.60,59,1,5.82,clear sky,0.0,0
1,1,Cape Town,ZA,-33.93,18.42,55.00,81,75,5.82,broken clouds,0.0,0
2,2,Bengkulu,ID,-3.80,102.27,75.34,86,100,4.16,overcast clouds,0.0,0
3,3,Punta Arenas,CL,-53.15,-70.92,33.80,86,75,9.17,broken clouds,0.0,0
4,4,Sao Lourenco Do Sul,BR,-31.37,-51.98,59.00,72,0,12.75,clear sky,0.0,0


In [4]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_question = str(input("Do you want it to be raining? (yes/no) "))
snow_question = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] == 0 if rain_question == "no" else city_data_df["Rain (inches)"] > 0)  & \
                                        (city_data_df["Snow (inches)"] == 0 if snow_question == "no" else city_data_df["Snow (inches)"] > 0) ]
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
2,2,Bengkulu,ID,-3.80,102.27,75.34,86,100,4.16,overcast clouds,0.0,0
14,14,Lumby,CA,50.25,-118.97,77.00,46,0,1.01,clear sky,0.0,0
15,15,Baleshwar,IN,21.49,86.93,82.81,82,100,11.03,overcast clouds,0.0,0
17,17,Hilo,US,19.73,-155.09,80.60,65,90,5.82,overcast clouds,0.0,0
19,19,Katsuura,JP,35.13,140.30,80.01,93,100,5.99,overcast clouds,0.0,0
26,26,Nishihara,JP,26.18,127.76,82.40,94,20,9.17,few clouds,0.0,0
34,34,Vila Do Maio,CV,15.13,-23.22,82.40,74,75,4.70,broken clouds,0.0,0
36,36,Kapaa,US,22.08,-159.32,82.99,74,20,13.87,few clouds,0.0,0
39,39,Kailua,US,21.40,-157.74,84.20,58,40,12.75,scattered clouds,0.0,0
45,45,Avarua,CK,-21.21,-159.78,77.00,78,44,5.82,scattered clouds,0.0,0


In [8]:
preferred_cities_df.count()

City_ID                157
City                   157
Country                157
Lat                    157
Lng                    157
Max Temp               157
Humidity               157
Cloudiness             157
Wind Speed             157
Current Description    157
Rain (inches)          157
Snow (inches)          157
dtype: int64

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,75.34,overcast clouds,-3.80,102.27,
14,Lumby,CA,77.00,clear sky,50.25,-118.97,
15,Baleshwar,IN,82.81,overcast clouds,21.49,86.93,
17,Hilo,US,80.60,overcast clouds,19.73,-155.09,
19,Katsuura,JP,80.01,overcast clouds,35.13,140.30,
26,Nishihara,JP,82.40,few clouds,26.18,127.76,
34,Vila Do Maio,CV,82.40,broken clouds,15.13,-23.22,
36,Kapaa,US,82.99,few clouds,22.08,-159.32,
39,Kailua,US,84.20,scattered clouds,21.40,-157.74,
45,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,


In [10]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [11]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bengkulu,ID,75.34,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
14,Lumby,CA,77.00,clear sky,50.25,-118.97,Twin Creeks Motel
15,Baleshwar,IN,82.81,overcast clouds,21.49,86.93,Banjara Hotels Pvt Ltd
17,Hilo,US,80.60,overcast clouds,19.73,-155.09,Hilo Hawaiian Hotel
19,Katsuura,JP,80.01,overcast clouds,35.13,140.30,Katsuura Hotel Mikazuki
26,Nishihara,JP,82.40,few clouds,26.18,127.76,Minshuku Agaihama
34,Vila Do Maio,CV,82.40,broken clouds,15.13,-23.22,Residence 'I Delfini'
36,Kapaa,US,82.99,few clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
39,Kailua,US,84.20,scattered clouds,21.40,-157.74,Sheffield House Bed & Breakfast
45,Avarua,CK,77.00,scattered clouds,-21.21,-159.78,Paradise Inn


In [14]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [19]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"